# Quantum Walk

Quantum walks is the quantum version of the classical algorithm [random walks](https://www.mit.edu/~kardar/teaching/projects/chemotaxis(AndreaSchmidt)/random.htm). In this notebook, we will focus on implementing a [discrete-time quantum walks](https://www.cs.umd.edu/~amchilds/qa/qa.pdf).

Consider a classical graph with $V$ set of vertices and $E$ set of edges. 
For example, we have a cycle with 4 nodes. Then, $V = \{0,1,2,3\} $ $E = \{(0,1),(1,0),(1,2),(2,1),(2,3),(3,2),(3,0),(0,3) \} $ 

The following will explain the quantum walk operator for the case of a circle with 4 nodes, and then the quantum walk operator for the case of a line with 16 nodes. Each section will go over parts of the implementation and a single cell to run the whole program:


## Circle with 4 nodes

![circle with 4 nodes](circle.png) 


### Encoding the vertices 

First, let's define the main method to map out the algorithm(Note that for main to work we need to run the other cells that contain the functions that we call in main). 

1. We encode the input state $\ket{j}$. This represents the vertices. We can use the QNum.
2. Let the Hilbert Space consist of states of the form $\ket{j,k}$ where $(j,k)\in E$. Note that $\ket{k}$ also encodes a vertex. This represents the adjacent vertices. We can also use QNum.



In [ ]:
from classiq import *
size = 2

@qfunc 
def main(vertices:Output[QNum], adjacent_vertices:Output[QNum]):

  allocate(size,vertices)
  hadamard_transform(vertices)
  allocate(size,adjacent_vertices)

  W_operator(vertices,adjacent_vertices)
  S_operator(vertices,adjacent_vertices)

### Evolution

3. Then, we have the evolution which contains 2 unitary operators. A step of the discrete-time quantum walk is described by these operators.

Let's start with the C operator which we will call W_operator in our code. 

$C := \sum_{j \in V} \ket{j}\bra{j} \otimes \left( 2\ket{\partial_j}\bra{\partial_j} - I \right)
 $


Assume access to $U_j \ket{0} = \ket{\partial_j}$

Then, $2\ket{\partial_j}\bra{\partial_j} - I  = U_j(2\ket{0}\bra{0} - I) U_j^{-1}$ 

Now, what is $(2\ket{0}\bra{0} - I)$ operator? If we apply it to $\ket{0}$:

$(2\ket{0}\bra{0} - I)\ket{0} = \ket{0}$. Notice that $\ket{0}$ is the eigenstate and we have an eigenvalue of $1$.

If we apply it to any other integer state that is NOT zero:

$(2\ket{0}\bra{0} - I)\ket{j \neq 0} = \ket{j \neq 0}$. Notice that $\ket{j \neq 0}$ is the eigenstate and we have an eigenvalue of $-1$.

This is exactly a quantum primitive - Phase kickback (PKB)!

$(2\ket{0}\bra{0} - I)\ket{x} = (-1)^{x \neq 0} \ket{x}$

The following implements the PKB (Note that this is just a subpart of the C operator):

In [83]:
@qfunc
def prepare_minus(x: QBit):
  X(x)
  H(x)
  
@qfunc
def zero_diffuzer(x: QNum):
  aux = QNum('aux')
  allocate(1,aux)
  within_apply(compute=lambda: prepare_minus(aux),
              action=lambda: diffuzer_oracle)
  

Now, let's implement the summation part. Since we have a summation of all vertices in the graph, we can use a for loop implement the summation over the vertices that we have.


In [90]:
@qfunc 
def W_operator(vertices:QNum, adjacent_vertices: QNum):
    for i in range(2**size):
      W_iteration(i,vertices,adjacent_vertices)

For each iteration of the W operator, we want to only apply the correspoding operator only if the state is $j$

In [95]:
def W_iteration(i:int,vertices: QNum, adjacent_vertices:QNum):
    prob = [0,0,0,0]
    prob[(i+1)% 4]=0.5
    prob[(i-1)% 4]=0.5
    print(f'State={i}, prob vec ={prob}')
    
    control(ctrl=vertices==i,
            operand=lambda: within_apply(
              compute= lambda: inplace_prepare_state(probabilities=prob, bound=0.01, target=adjacent_vertices),
              action= lambda: zero_diffuzer(adjacent_vertices)))

Now, let's implement the S operator.

$ S := \sum_{(j,k) \in E} \ket{j,k}\bra{k,j}  $

Here we see a summation of all the adjacent vertices then we want to swap. We will now design an oracle that gives us this condition. We expect the result to be 1 if the current vertex is connected adjacent vertex is a part of our graph. Otherwise, the result would be zero because there is no edge between them.

In [ ]:
@qfunc
def edge_oracle(res:Output[QBit], vertices: QNum, adjacent_vertices: QNum):
  res |= (((vertices+adjacent_vertices)%2) ==1)

Here is the program all together for a quantum walk on a circle with 4 nodes:

In [2]:
from classiq import *
size = 2

@qfunc
def prepare_minus(x: QBit):
  X(x)
  H(x)


@qfunc
def diffuzer_oracle(aux: Output[QNum],x:QNum):
  aux^=(x!=0)


@qfunc
def zero_diffuzer(x: QNum):
  aux = QNum('aux')
  allocate(1,aux)
  within_apply(compute=lambda: prepare_minus(aux),
              action=lambda: diffuzer_oracle)


def W_iteration(i:int,vertices: QNum, adjacent_vertices:QNum):
    prob = [0,0,0,0]
    prob[(i+1)% 4]=0.5
    prob[(i-1)% 4]=0.5
    print(f'State={i}, prob vec ={prob}')
    
    control(ctrl=vertices==i,
            operand=lambda: within_apply(
              compute= lambda: inplace_prepare_state(probabilities=prob, bound=0.01, target=adjacent_vertices),
              action= lambda: zero_diffuzer(adjacent_vertices)))


@qfunc 
def W_operator(vertices:QNum, adjacent_vertices: QNum):
    for i in range(2**size):
      W_iteration(i,vertices,adjacent_vertices)


@qfunc
def edge_oracle(res:Output[QBit], vertices: QNum, adjacent_vertices: QNum):
  res |= (((vertices+adjacent_vertices)%2) ==1)


@qfunc 
def bitwise_swap(x: QArray[QBit], y:QArray[QBit]):
  repeat(count= x.len,
    iteration= lambda i: SWAP(x[i],y[i]))
  

@qfunc 
def S_operator(vertices:QNum, adjacent_vertices: QNum):
    res = QNum('res')
    edge_oracle(res,vertices,adjacent_vertices)
    control(ctrl= res==1,
        operand= lambda: bitwise_swap(vertices,adjacent_vertices))

@qfunc 
def main(vertices:Output[QNum], adjacent_vertices:Output[QNum]):

  allocate(size,vertices)
  hadamard_transform(vertices)
  allocate(size,adjacent_vertices)

  W_operator(vertices,adjacent_vertices)
  S_operator(vertices,adjacent_vertices)

qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

State=0, prob vec =[0, 0.5, 0, 0.5]
State=1, prob vec =[0.5, 0, 0.5, 0]
State=2, prob vec =[0, 0.5, 0, 0.5]
State=3, prob vec =[0.5, 0, 0.5, 0]
Opening: https://platform.classiq.io/circuit/a5400c6f-ef79-4ea5-aa4b-b53e74cd0156?version=0.43.3


## Line with 16 nodes

![line with 16 nodes](line.png) 
